In [1]:
pip install transformers tensorflow

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
import torch

# Load tokenizer and model
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 1. Preprocess and Create Dataset
def load_dataset(text, tokenizer, max_length=512):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    return inputs.input_ids

# Example data, you may replace this with a larger corpus
text_data = """
Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods.
One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her.
"""

# Encode the text
dataset = load_dataset(text_data, tokenizer)

# 2. Set Up Training Arguments
def train_model(num_epochs):
    training_args = TrainingArguments(
        output_dir='./results',
        overwrite_output_dir=True,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=1,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='./logs',
        logging_steps=200,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    # Train the model
    trainer.train()

# 3. Train the model with different epoch counts
for epoch_count in [20, 60, 70]:
    print(f"Training with {epoch_count} epochs...")
    train_model(epoch_count)

# 4. Text Generation
def generate_text(seed_text):
    inputs = tokenizer(seed_text, return_tensors="pt")
    output = model.generate(
        inputs["input_ids"], max_length=100, num_return_sequences=1, no_repeat_ngram_size=2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Seed text for generation
seed_text = "Once upon a time, there was"
generated_text = generate_text(seed_text)
print("Generated Text:", generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Step,Training Loss


Training with 70 epochs...


Step,Training Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. 
One day, her mother asked her to take a basket of goodies to her grandparents. On her way through the forest, she met a big bad wolf who wanted to eat her.
On her first day of the trip,she met her big brother who asked to be with her on the way.
